In [47]:
import polars as pl
print(pl.__version__)

0.20.31


### Actors and Directors Who Cooperated At Least Three Times

#### Question

DataFrame: ActorDirector

| Column Name | Type    |
|:-----------:|:-------:|
| actor_id    | int     |
| director_id | int     |
| timestamp   | int     |

timestamp is the primary key (column with unique values) for this table.
 

Write a solution to find all the pairs (actor_id, director_id) where the actor has cooperated with the director at least three times.

Return the result table in any order.

The result format is in the following example.

Input:<br>
ActorDirector dataframe:

| actor_id    | director_id | timestamp   |
|:-----------:|:-----------:|:-----------:|
| 1           | 1           | 0           |
| 1           | 1           | 1           |
| 1           | 1           | 2           |
| 1           | 2           | 3           |
| 1           | 2           | 4           |
| 2           | 1           | 5           |
| 2           | 1           | 6           |

Output: 

| actor_id    | director_id |
|:-----------:|:-----------:|
| 1           | 1           |

Explanation: The only pair is (1, 1) where they cooperated exactly 3 times.

#### Testcase

In [48]:
# Test data
data = [[1, 1, 0], [1, 1, 1], [1, 1, 2], [1, 2, 3], [1, 2, 4], [2, 1, 5], [2, 1, 6]]


# Create the DataFrame
actor_director = pl.DataFrame(
    data,
    schema=['actor_id', 'director_id', 'timestamp']
)

# Display the DataFrame
print(actor_director)

shape: (7, 3)
┌──────────┬─────────────┬───────────┐
│ actor_id ┆ director_id ┆ timestamp │
│ ---      ┆ ---         ┆ ---       │
│ i64      ┆ i64         ┆ i64       │
╞══════════╪═════════════╪═══════════╡
│ 1        ┆ 1           ┆ 0         │
│ 1        ┆ 1           ┆ 1         │
│ 1        ┆ 1           ┆ 2         │
│ 1        ┆ 2           ┆ 3         │
│ 1        ┆ 2           ┆ 4         │
│ 2        ┆ 1           ┆ 5         │
│ 2        ┆ 1           ┆ 6         │
└──────────┴─────────────┴───────────┘


#### Solution

In [49]:
def actors_and_directors(actor_director: pl.DataFrame) -> pl.DataFrame:
    
    # Group by 'actor_id' and 'director_id', and count the occurrences
    freq_act = (
        actor_director
        .group_by(['actor_id', 'director_id'])
        .agg(pl.col('director_id').count().alias('count'))
    )
    
    # Filter the rows where the count is greater than or equal to 3
    result = freq_act.filter(pl.col('count') >= 3).select(['actor_id', 'director_id'])
    
    return result

# Display the result
print(actors_and_directors(actor_director=actor_director))

shape: (1, 2)
┌──────────┬─────────────┐
│ actor_id ┆ director_id │
│ ---      ┆ ---         │
│ i64      ┆ i64         │
╞══════════╪═════════════╡
│ 1        ┆ 1           │
└──────────┴─────────────┘


### Replace Employee ID With The Unique Identifier

#### Question

DataFrame: Employees

| Column Name   | Type    |
|:-------------:|:-------:|
| id            | int     |
| name          | varchar |

id is the primary key (column with unique values) for this table.<br>
Each row of this table contains the id and the name of an employee in a company.
 

DataFrame: EmployeeUNI

| Column Name   | Type    |
|:-------------:|:-------:|
| id            | int     |
| unique_id     | int     |

(id, unique_id) is the primary key (combination of columns with unique values) for this table.<br>
Each row of this table contains the id and the corresponding unique id of an employee in the company.
 

Write a solution to show the unique ID of each user, If a user does not have a unique ID replace just show null.

Return the result table in any order.

The result format is in the following example.

Input:<br>
Employees dataframe:

| id | name     |
|:--:|:--------:|
| 1  | Alice    |
| 7  | Bob      |
| 11 | Meir     |
| 90 | Winston  |
| 3  | Jonathan |

EmployeeUNI dataframe:

| id | unique_id |
|:--:|:---------:|
| 3  | 1         |
| 11 | 2         |
| 90 | 3         |

Output: 

| unique_id | name     |
|:---------:|:--------:|
| null      | Alice    |
| null      | Bob      |
| 2         | Meir     |
| 3         | Winston  |
| 1         | Jonathan |

Explanation:<br>
Alice and Bob do not have a unique ID, We will show null instead.<br>
The unique ID of Meir is 2.<br>
The unique ID of Winston is 3.<br>
The unique ID of Jonathan is 1.

#### Testcase

In [50]:
# Test data
data_emp = [[1, 'Alice'], [7, 'Bob'], [11, 'Meir'], [90, 'Winston'], [3, 'Jonathan']]
data_uni = [[3, 1], [11, 2], [90, 3]]


# Create the DataFrame
employees = pl.DataFrame(
    data_emp,
    schema=['id', 'name']
)

employee_uni = pl.DataFrame(
    data_uni,
    schema=['id', 'unique_id']
)

# Display the DataFrame
print('employee df:', employees)
print('employee_uni df', employee_uni)

employee df: shape: (5, 2)
┌─────┬──────────┐
│ id  ┆ name     │
│ --- ┆ ---      │
│ i64 ┆ str      │
╞═════╪══════════╡
│ 1   ┆ Alice    │
│ 7   ┆ Bob      │
│ 11  ┆ Meir     │
│ 90  ┆ Winston  │
│ 3   ┆ Jonathan │
└─────┴──────────┘
employee_uni df shape: (3, 2)
┌─────┬───────────┐
│ id  ┆ unique_id │
│ --- ┆ ---       │
│ i64 ┆ i64       │
╞═════╪═══════════╡
│ 3   ┆ 1         │
│ 11  ┆ 2         │
│ 90  ┆ 3         │
└─────┴───────────┘


#### Solution

In [51]:
def replace_employee_id(employees: pl.DataFrame, employee_uni: pl.DataFrame) -> pl.DataFrame:
    
    # Perform a left join on 'id' column and select 'unique_id' and 'name' columns
    result = employees.join(employee_uni, on='id', how='left', coalesce=True).select(['unique_id', 'name'])
    
    return result

# Display the result
print(replace_employee_id(employees=employees, employee_uni=employee_uni))

shape: (5, 2)
┌───────────┬──────────┐
│ unique_id ┆ name     │
│ ---       ┆ ---      │
│ i64       ┆ str      │
╞═══════════╪══════════╡
│ null      ┆ Alice    │
│ null      ┆ Bob      │
│ 2         ┆ Meir     │
│ 3         ┆ Winston  │
│ 1         ┆ Jonathan │
└───────────┴──────────┘


### Students and Examinations

#### Question

DataFrame: Students

| Column Name   | Type    |
|:-------------:|:-------:|
| student_id    | int     |
| student_name  | varchar |

student_id is the primary key (column with unique values) for this table.<br>
Each row of this table contains the ID and the name of one student in the school.
 

Table: Subjects

| Column Name  | Type    |
|:------------:|:-------:|
| subject_name | varchar |

subject_name is the primary key (column with unique values) for this table.<br>
Each row of this table contains the name of one subject in the school.
 

Table: Examinations

| Column Name  | Type    |
|:------------:|:-------:|
| student_id   | int     |
| subject_name | varchar |

There is no primary key (column with unique values) for this table. It may contain duplicates.<br>
Each student from the Students table takes every course from the Subjects table.<br>
Each row of this table indicates that a student with ID student_id attended the exam of subject_name.

Write a solution to find the number of times each student attended each exam.

Return the result table ordered by student_id and subject_name.

The result format is in the following example.

Input:<br>
Students dataframe:

| student_id | student_name |
|:----------:|:------------:|
| 1          | Alice        |
| 2          | Bob          |
| 13         | John         |
| 6          | Alex         |

Subjects dataframe:

| subject_name |
|:------------:|
| Math         |
| Physics      |
| Programming  |

Examinations dataframe:

| student_id | subject_name |
|:----------:|:------------:|
| 1          | Math         |
| 1          | Physics      |
| 1          | Programming  |
| 2          | Programming  |
| 1          | Physics      |
| 1          | Math         |
| 13         | Math         |
| 13         | Programming  |
| 13         | Physics      |
| 2          | Math         |
| 1          | Math         |

Output: 

| student_id | student_name | subject_name | attended_exams |
|:----------:|:------------:|:------------:|:--------------:|
| 1          | Alice        | Math         | 3              |
| 1          | Alice        | Physics      | 2              |
| 1          | Alice        | Programming  | 1              |
| 2          | Bob          | Math         | 1              |
| 2          | Bob          | Physics      | 0              |
| 2          | Bob          | Programming  | 1              |
| 6          | Alex         | Math         | 0              |
| 6          | Alex         | Physics      | 0              |
| 6          | Alex         | Programming  | 0              |
| 13         | John         | Math         | 1              |
| 13         | John         | Physics      | 1              |
| 13         | John         | Programming  | 1              |

Explanation:<br>
The result table should contain all students and all subjects.<br>
Alice attended the Math exam 3 times, the Physics exam 2 times, and the Programming exam 1 time.<br>
Bob attended the Math exam 1 time, the Programming exam 1 time, and did not attend the Physics exam.<br>
Alex did not attend any exams.<br>
John attended the Math exam 1 time, the Physics exam 1 time, and the Programming exam 1 time.

#### Testcase

In [52]:
# Test data
data_stu = [[1, 'Alice'], [2, 'Bob'], [13, 'John'], [6, 'Alex']]
data_sub = [['Math'], ['Physics'], ['Programming']]
data_ex = [[1, 'Math'], [1, 'Physics'], [1, 'Programming'], [2, 'Programming'], [1, 'Physics'], [1, 'Math'], [13, 'Math'], [13, 'Programming'], [13, 'Physics'], [2, 'Math'], [1, 'Math']]


# Create the DataFrame
students = pl.DataFrame(
    data_stu,
    schema=['student_id', 'student_name']
)

subjects = pl.DataFrame(
    data_sub,
    schema=['subject_name']
)

examinations = pl.DataFrame(
    data_ex,
    schema=['student_id', 'subject_name']
)

# Display the DataFrame
print('students df:', students)
print('subjects df', subjects)
print('examinations df', examinations)

students df: shape: (4, 2)
┌────────────┬──────────────┐
│ student_id ┆ student_name │
│ ---        ┆ ---          │
│ i64        ┆ str          │
╞════════════╪══════════════╡
│ 1          ┆ Alice        │
│ 2          ┆ Bob          │
│ 13         ┆ John         │
│ 6          ┆ Alex         │
└────────────┴──────────────┘
subjects df shape: (3, 1)
┌──────────────┐
│ subject_name │
│ ---          │
│ str          │
╞══════════════╡
│ Math         │
│ Physics      │
│ Programming  │
└──────────────┘
examinations df shape: (11, 2)
┌────────────┬──────────────┐
│ student_id ┆ subject_name │
│ ---        ┆ ---          │
│ i64        ┆ str          │
╞════════════╪══════════════╡
│ 1          ┆ Math         │
│ 1          ┆ Physics      │
│ 1          ┆ Programming  │
│ 2          ┆ Programming  │
│ 1          ┆ Physics      │
│ …          ┆ …            │
│ 13         ┆ Math         │
│ 13         ┆ Programming  │
│ 13         ┆ Physics      │
│ 2          ┆ Math         │
│ 1          

#### Solution

In [54]:
def students_and_examinations(students: pl.DataFrame, subjects: pl.DataFrame, examinations: pl.DataFrame) -> pl.DataFrame:
    
    # Perform a cross join to get all combinations of students and subjects
    cartesian_product = students.join(subjects, how='cross')
    
    # Group by 'student_id' and 'subject_name', and count the number of attended exams for each student and subject
    exam_counts = (
        examinations
        .group_by(['student_id', 'subject_name'])
        .agg(pl.col('student_id').count().alias('attended_exams'))
    )

    # Perform a left join to retain all combinations of students and subjects, and fill missing values with 0
    result = cartesian_product.join(exam_counts, on=['student_id', 'subject_name'], how='left', coalesce=True).fill_null(0)

    return result.sort(['student_id', 'subject_name'])

# Display the result
print(students_and_examinations(students=students, subjects=subjects, examinations=examinations))

shape: (12, 4)
┌────────────┬──────────────┬──────────────┬────────────────┐
│ student_id ┆ student_name ┆ subject_name ┆ attended_exams │
│ ---        ┆ ---          ┆ ---          ┆ ---            │
│ i64        ┆ str          ┆ str          ┆ i64            │
╞════════════╪══════════════╪══════════════╪════════════════╡
│ 1          ┆ Alice        ┆ Math         ┆ 3              │
│ 1          ┆ Alice        ┆ Physics      ┆ 2              │
│ 1          ┆ Alice        ┆ Programming  ┆ 1              │
│ 2          ┆ Bob          ┆ Math         ┆ 1              │
│ 2          ┆ Bob          ┆ Physics      ┆ 0              │
│ …          ┆ …            ┆ …            ┆ …              │
│ 6          ┆ Alex         ┆ Physics      ┆ 0              │
│ 6          ┆ Alex         ┆ Programming  ┆ 0              │
│ 13         ┆ John         ┆ Math         ┆ 1              │
│ 13         ┆ John         ┆ Physics      ┆ 1              │
│ 13         ┆ John         ┆ Programming  ┆ 1         

### Managers with at Least 5 Direct Reports

#### Question

DataFrame: Employee

| Column Name | Type    |
|:-----------:|:-------:|
| id          | int     |
| name        | varchar |
| department  | varchar |
| managerId   | int     |

id is the primary key (column with unique values) for this table.<br>
Each row of this table indicates the name of an employee, their department, and the id of their manager.<br>
If managerId is null, then the employee does not have a manager.<br>
No employee will be the manager of themself.
 

Write a solution to find managers with at least five direct reports.

Return the result table in any order.

The result format is in the following example.

Input:<br>
Employee dataframe:

| id  | name  | department | managerId |
|:---:|:-----:|:----------:|:---------:|
| 101 | John  | A          | null      |
| 102 | Dan   | A          | 101       |
| 103 | James | A          | 101       |
| 104 | Amy   | A          | 101       |
| 105 | Anne  | A          | 101       |
| 106 | Ron   | B          | 101       |

Output: 

| name |
|:----:|
| John |

#### Testcase

In [55]:
# Test data
data = [[101, 'John', 'A', None], [102, 'Dan', 'A', 101], [103, 'James', 'A', 101], [104, 'Amy', 'A', 101], [105, 'Anne', 'A', 101], [106, 'Ron', 'B', 101]]

# Create the DataFrame
employee = pl.DataFrame(
    data,
    schema=['id', 'name', 'department', 'managerId']
)

# Display the DataFrame
print(employee)

shape: (6, 4)
┌─────┬───────┬────────────┬───────────┐
│ id  ┆ name  ┆ department ┆ managerId │
│ --- ┆ ---   ┆ ---        ┆ ---       │
│ i64 ┆ str   ┆ str        ┆ i64       │
╞═════╪═══════╪════════════╪═══════════╡
│ 101 ┆ John  ┆ A          ┆ null      │
│ 102 ┆ Dan   ┆ A          ┆ 101       │
│ 103 ┆ James ┆ A          ┆ 101       │
│ 104 ┆ Amy   ┆ A          ┆ 101       │
│ 105 ┆ Anne  ┆ A          ┆ 101       │
│ 106 ┆ Ron   ┆ B          ┆ 101       │
└─────┴───────┴────────────┴───────────┘


#### Solution

In [56]:
def find_managers(employee: pl.DataFrame) -> pl.DataFrame:
    
    # Group by 'managerId' and count the number of direct reports
    direct_reports_count = (
        employee
        .group_by('managerId')
        .agg(pl.col('id').count().alias('report_count'))
    )

    # Filter managers with five or more direct reports
    managers_with_five_reports = direct_reports_count.filter(pl.col('report_count') >= 5)

    # Perform an inner join between filtered managers and employee DataFrame to get manager details
    manager = (
        managers_with_five_reports
        .join(employee, left_on='managerId', right_on='id')
    )

    # Select only the 'name' column
    result = manager.select('name')

    return result

# Display the result
print(find_managers(employee=employee))

shape: (1, 1)
┌──────┐
│ name │
│ ---  │
│ str  │
╞══════╡
│ John │
└──────┘


### Sales Person

#### Question

DataFrame: SalesPerson

| Column Name     | Type    |
|:---------------:|:-------:|
| sales_id        | int     |
| name            | varchar |
| salary          | int     |
| commission_rate | int     |
| hire_date       | date    |

sales_id is the primary key (column with unique values) for this table.<br>
Each row of this table indicates the name and the ID of a salesperson alongside their salary, commission rate, and hire date.
 

DataFrame: Company

| Column Name | Type    |
|:-----------:|:-------:|
| com_id      | int     |
| name        | varchar |
| city        | varchar |

com_id is the primary key (column with unique values) for this table.<br>
Each row of this table indicates the name and the ID of a company and the city in which the company is located.
 

DataFrame: Orders

| Column Name | Type |
|:-----------:|:----:|
| order_id    | int  |
| order_date  | date |
| com_id      | int  |
| sales_id    | int  |
| amount      | int  |

order_id is the primary key (column with unique values) for this table.<br>
com_id is a foreign key (reference column) to com_id from the Company table.<br>
sales_id is a foreign key (reference column) to sales_id from the SalesPerson table.<br>
Each row of this table contains information about one order. This includes the ID of the company, the ID of the salesperson, the date of the order, and the amount paid.


Write a solution to find the names of all the salespersons who did not have any orders related to the company with the name "RED".

Return the result table in any order.

The result format is in the following example.

Input:<br>
SalesPerson dataframe:

| sales_id | name | salary | commission_rate | hire_date  |
|:--------:|:----:|:------:|:---------------:|:----------:|
| 1        | John | 100000 | 6               | 4/1/2006   |
| 2        | Amy  | 12000  | 5               | 5/1/2010   |
| 3        | Mark | 65000  | 12              | 12/25/2008 |
| 4        | Pam  | 25000  | 25              | 1/1/2005   |
| 5        | Alex | 5000   | 10              | 2/3/2007   |

Company dataframe:

| com_id | name   | city     |
|:------:|:------:|:--------:|
| 1      | RED    | Boston   |
| 2      | ORANGE | New York |
| 3      | YELLOW | Boston   |
| 4      | GREEN  | Austin   |

Orders dataframe:

| order_id | order_date | com_id | sales_id | amount |
|:--------:|:----------:|:------:|:--------:|:------:|
| 1        | 1/1/2014   | 3      | 4        | 10000  |
| 2        | 2/1/2014   | 4      | 5        | 5000   |
| 3        | 3/1/2014   | 1      | 1        | 50000  |
| 4        | 4/1/2014   | 1      | 4        | 25000  |

Output: 

| name |
|:----:|
| Amy  |
| Mark |
| Alex |

Explanation:<br>
According to orders 3 and 4 in the Orders table, it is easy to tell that only salesperson John and Pam have sales to company RED, so we report all the other names in the table salesperson.

#### Testcase

In [57]:
# Test data
data_sp = [[1, 'John', 100000, 6, '4/1/2006'], [2, 'Amy', 12000, 5, '5/1/2010'], [3, 'Mark', 65000, 12, '12/25/2008'], [4, 'Pam', 25000, 25, '1/1/2005'], [5, 'Alex', 5000, 10, '2/3/2007']]
data_company = [[1, 'RED', 'Boston'], [2, 'ORANGE', 'New York'], [3, 'YELLOW', 'Boston'], [4, 'GREEN', 'Austin']]
data_orders = [[1, '1/1/2014', 3, 4, 10000], [2, '2/1/2014', 4, 5, 5000], [3, '3/1/2014', 1, 1, 50000], [4, '4/1/2014', 1, 4, 25000]]


# Create the DataFrame
sales_personnel = pl.DataFrame(
    data_sp,
    schema=['sales_id', 'name', 'salary', 'commission_rate', 'hire_date']
)

company = pl.DataFrame(
    data_company,
    schema=['com_id', 'name', 'city']
)

orders = pl.DataFrame(
    data_orders,
    schema=['order_id', 'order_date', 'com_id', 'sales_id', 'amount']
)

# Display the DataFrame
print('sales_person df:', sales_person)
print('company df', company)
print('orders df', orders)

sales_person df: <function sales_person at 0x11c6f6f20>
company df shape: (4, 3)
┌────────┬────────┬──────────┐
│ com_id ┆ name   ┆ city     │
│ ---    ┆ ---    ┆ ---      │
│ i64    ┆ str    ┆ str      │
╞════════╪════════╪══════════╡
│ 1      ┆ RED    ┆ Boston   │
│ 2      ┆ ORANGE ┆ New York │
│ 3      ┆ YELLOW ┆ Boston   │
│ 4      ┆ GREEN  ┆ Austin   │
└────────┴────────┴──────────┘
orders df shape: (4, 5)
┌──────────┬────────────┬────────┬──────────┬────────┐
│ order_id ┆ order_date ┆ com_id ┆ sales_id ┆ amount │
│ ---      ┆ ---        ┆ ---    ┆ ---      ┆ ---    │
│ i64      ┆ str        ┆ i64    ┆ i64      ┆ i64    │
╞══════════╪════════════╪════════╪══════════╪════════╡
│ 1        ┆ 1/1/2014   ┆ 3      ┆ 4        ┆ 10000  │
│ 2        ┆ 2/1/2014   ┆ 4      ┆ 5        ┆ 5000   │
│ 3        ┆ 3/1/2014   ┆ 1      ┆ 1        ┆ 50000  │
│ 4        ┆ 4/1/2014   ┆ 1      ┆ 4        ┆ 25000  │
└──────────┴────────────┴────────┴──────────┴────────┘


### Solution

In [58]:
def sales_person(sales_personnel: pl.DataFrame, company: pl.DataFrame, orders: pl.DataFrame) -> pl.DataFrame:
    
    # Merge DataFrames to find salespersons with orders related to "RED" companies
    merged_df = orders.join(company, on='com_id')
    red_sales_ids = merged_df.filter(pl.col('name') == 'RED')['sales_id'].unique()

    # Filter salespersons who do not have orders related to "RED" companies
    result = sales_personnel.filter(~pl.col('sales_id').is_in(red_sales_ids)).select('name')
    
    return result

# Display the result
print(sales_person(sales_personnel=sales_personnel, company=company, orders=orders))


shape: (3, 1)
┌──────┐
│ name │
│ ---  │
│ str  │
╞══════╡
│ Amy  │
│ Mark │
│ Alex │
└──────┘
